In [6]:
!git pull
!pip install transformers

Already up-to-date.


In [2]:
import os
import boto3
import re
import copy
import time
import io
import struct
from time import gmtime, strftime
from sagemaker import get_execution_role
import torch
from transformers import SquadV1Processor, squad_convert_examples_to_features, BertTokenizer

S3_BUCKET_NAME = "2020-05-gym-bert"

In [3]:
!curl https://raw.githubusercontent.com/rajpurkar/SQuAD-explorer/master/dataset/train-v1.1.json --output ./dataset/train-v1.1.json
!curl https://raw.githubusercontent.com/rajpurkar/SQuAD-explorer/master/dataset/dev-v1.1.json --output ./dataset/dev-v1.1.json
boto3.Session().resource('s3').Bucket(S3_BUCKET_NAME).Object('SQuAD_train-v1.1.json').upload_file('./dataset/train-v1.1.json')
boto3.Session().resource('s3').Bucket(S3_BUCKET_NAME).Object('SQuAD_dev-v1.1.json').upload_file('./dataset/dev-v1.1.json')

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 28.8M  100 28.8M    0     0  11.0M      0  0:00:02  0:00:02 --:--:-- 11.0M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 4740k  100 4740k    0     0  2411k      0  0:00:01  0:00:01 --:--:-- 2410k


In [4]:
#tokenizer = torch.hub.load('huggingface/pytorch-transformers', 'tokenizer', 'bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained("bert-base-cased")
processor = SquadV1Processor()
examples = processor.get_train_examples('.', './dataset/dev-v1.1.json')
features, dataset = squad_convert_examples_to_features(
            examples=examples,
            tokenizer=tokenizer,
            max_seq_length=384,
            doc_stride=128,
            max_query_length=64,
            is_training=True,
            return_dataset="pt",
            threads=1,
        )

add example index and unique id: 100%|██████████| 10570/10570 [00:00<00:00, 502314.70it/s]
